In [1]:
import numpy
import netCDF4
from osgeo import gdal, osr
#from datetime import datetime, date
import datetime
import os

siloDIR = 'P:/projects/SILO/Wes/mwet/'
baseOutputDIR = 'C:/DDrive/SILO/tifs/mwet'
#baseOutputDIR = 'P:/projects/SILO/mwet44_posHeight'
#baseOutputDIR = 'P:/projects/SILO/Wes/tifs/mwet'
#baseOutputDIR = 'P:/projects/SILO/mwet10_negHeightRevArray'
#dataVarName = 'daily_rain'
dataVarName = 'et_morton_wet'
fileNameEndBit = '_mwet.tif'
#fileNameEndBit = '_rain.tif'

startYear = 1980
endYear = 2019

if not os.path.exists(baseOutputDIR):
    os.makedirs(baseOutputDIR)



In [2]:

# Loop over years
#for year in range(1961, 1990):
# range uses start and STOP values, so last value isn't processed
#for year in range(2000, 2002):
for year in range(startYear, endYear + 1):
    print('Current year: ' + str(year))
    
    #create necessary dir for this year
    thisYearDir = baseOutputDIR + '/' + str(year)
    if not os.path.exists(thisYearDir):
        os.makedirs(thisYearDir)
    
    theFile = siloDIR + '{:d}.'.format(year) + dataVarName + '.nc'
    #with netCDF4.Dataset('{:d}.monthly_rain.nc'.format(year), 'r') as dataset:
    print('Processing: ' + theFile)
    #with netCDF4.Dataset(siloDIR + '{:d}.'.format(year) + dataVarName + '.nc', 'r') as theDataSet:
    
    theDataSet = gdal.Open(theFile)
    
    #print( "Driver: ",theDataSet.GetDriver().ShortName, theDataSet.GetDriver().LongName)
    #print( "Size is ", theDataSet.RasterXSize, theDataSet.RasterYSize)
    #print( "Bands = ", theDataSet.RasterCount)
    #print( "Coordinate System is:", theDataSet.GetProjectionRef ())
    #print( "GetGeoTransform() = ", theDataSet.GetGeoTransform ())
        
    #Need some vals for creating output extent etc
    #The GeoTransform object is:
    
    #GeoTransform[0] /* top left x */
    #GeoTransform[1] /* w-e pixel resolution */
    #GeoTransform[2] /* 0 */
    #GeoTransform[3] /* top left y */
    #GeoTransform[4] /* 0 */
    #GeoTransform[5] /* n-s pixel resolution (negative value) */
    
    #zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))
    #y_max is the SOUTHERN value! For us, that's MIN! Oh well...
    
    netCDFData = netCDF4.Dataset(theFile, 'r')
    #print(netCDFData)
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    #These need to be adjusted so that we get pixel edges shifted, leaving
    #our SILO coords mid-pixel
    pixWidth = 0.05
    
    #Source Gdal loading looks at this pixHeight value to work out the lower left coordinate
    #Either way, Source will interpret inGeoTransform[3] as being our North coordinate,
    #calculating the yllCorner to be a MORE NEGATIVE value.
    
    #xllCorner = inGeoTransform[0];
    #yllCorner = (inGeoTransform[5] > 0)?inGeoTransform[3] - ( inGeoTransform[5] * inDataset.RasterYSize ):
    #                                   inGeoTransform[3] + ( inGeoTransform[5] * inDataset.RasterYSize );
    
    pixHeight = pixWidth * -1
    #pixHeight = pixWidth
    
    #x_min = lons[:1] - (0.5 * pixWidth)
    x_min = min(lons) - (0.5 * pixWidth)
    
    #This gives first value, which in SILO is the smallest or 'most negative' or 'most southerly'
    #y_min = lats[:1] - (0.5 * pixHeight)
    
    #This gives last value, which in SILO is the largest or 'most positive' or 'most northerly'
    #y_min = lats[len(lats)-1:] - (0.5 * pixHeight)
    
    #-10
    y_min = max(lats) - (0.5 * pixHeight)
    
    #-44
    #yes, use lower left if supplying positive pixel height
    #y_min = min(lats) - (0.5 * pixHeight)
    
    
    netCDFData.close()
    
    dStart = datetime.date(int(year), 1, 1)
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)#WGS84

    theDataSet.SetProjection(srs.ExportToWkt())
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    for band in range( theDataSet.RasterCount ):
           
        dateAdd = band
        dateNow = dStart + datetime.timedelta(days=dateAdd)
        
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        
        band += 1
        #print("[ GETTING BAND ]: ", band)
        srcband = theDataSet.GetRasterBand(band)
        
        if srcband is None:
            continue
        
        scaleFactor = 1.0
        
        sf = srcband.GetMetadataItem(dataVarName + '_scale_factor')
        nd = srcband.GetMetadataItem(dataVarName + '__FillValue')
        #print(sf)
        
        scaleFactor = float(sf)
        rawNoData = float(nd)
        
        rasterOutputFile = thisYearDir + '/' + dateForFileName + fileNameEndBit
        outRaster = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, theDataSet.RasterXSize, theDataSet.RasterYSize, 1, gdal.GDT_Float32)
        outRaster.SetGeoTransform((x_min, pixWidth, 0, y_min, 0, pixHeight))
        outBand = outRaster.GetRasterBand(1)
        bandArray = srcband.ReadAsArray()
        
        #Why does gdal_grid turn image upside-down?
        #revert the array lines
        #A few things to note:
        #
        #The point used in the geotransform should be upper-left
        #The resolution in y-direction should be negative
        #In the projection (at least the ones I use) positive y-direction is up
        #In numpy arrays positive y-direction is down
        #When using gdal's WriteArray it uses the upper left corner
        #
        #Hope this helps other people's confusion.
        
        bandArray = bandArray[::-1,:]
        outBand.WriteArray(bandArray * scaleFactor)
        outBand.SetNoDataValue(rawNoData * scaleFactor)
        outRaster.SetProjection(srs.ExportToWkt())
        outBand.FlushCache()
    
    

print("All Done")

Current year: 1980
Processing: P:/projects/SILO/Wes/mwet/1980.et_morton_wet.nc
Current year: 1981
Processing: P:/projects/SILO/Wes/mwet/1981.et_morton_wet.nc
Current year: 1982
Processing: P:/projects/SILO/Wes/mwet/1982.et_morton_wet.nc
Current year: 1983
Processing: P:/projects/SILO/Wes/mwet/1983.et_morton_wet.nc
Current year: 1984
Processing: P:/projects/SILO/Wes/mwet/1984.et_morton_wet.nc
Current year: 1985
Processing: P:/projects/SILO/Wes/mwet/1985.et_morton_wet.nc
Current year: 1986
Processing: P:/projects/SILO/Wes/mwet/1986.et_morton_wet.nc
Current year: 1987
Processing: P:/projects/SILO/Wes/mwet/1987.et_morton_wet.nc
Current year: 1988
Processing: P:/projects/SILO/Wes/mwet/1988.et_morton_wet.nc
Current year: 1989
Processing: P:/projects/SILO/Wes/mwet/1989.et_morton_wet.nc
Current year: 1990
Processing: P:/projects/SILO/Wes/mwet/1990.et_morton_wet.nc
Current year: 1991
Processing: P:/projects/SILO/Wes/mwet/1991.et_morton_wet.nc
Current year: 1992
Processing: P:/projects/SILO/Wes/

In [3]:
siloDIR = 'P:/projects/SILO/Wes/rain/'
baseOutputDIR = 'C:/DDrive/SILO/tifs/rain'
dataVarName = 'daily_rain'
fileNameEndBit = '_rain.tif'

startYear = 1980
endYear = 2019

if not os.path.exists(baseOutputDIR):
    os.makedirs(baseOutputDIR)

# Loop over years
#for year in range(1961, 1990):
# range uses start and STOP values, so last value isn't processed
#for year in range(2000, 2002):
for year in range(startYear, endYear + 1):
    print('Current year: ' + str(year))
    
    #create necessary dir for this year
    thisYearDir = baseOutputDIR + '/' + str(year)
    if not os.path.exists(thisYearDir):
        os.makedirs(thisYearDir)
    
    theFile = siloDIR + '{:d}.'.format(year) + dataVarName + '.nc'
    #with netCDF4.Dataset('{:d}.monthly_rain.nc'.format(year), 'r') as dataset:
    print('Processing: ' + theFile)
    #with netCDF4.Dataset(siloDIR + '{:d}.'.format(year) + dataVarName + '.nc', 'r') as theDataSet:
    
    theDataSet = gdal.Open(theFile)
    
    #print( "Driver: ",theDataSet.GetDriver().ShortName, theDataSet.GetDriver().LongName)
    #print( "Size is ", theDataSet.RasterXSize, theDataSet.RasterYSize)
    #print( "Bands = ", theDataSet.RasterCount)
    #print( "Coordinate System is:", theDataSet.GetProjectionRef ())
    #print( "GetGeoTransform() = ", theDataSet.GetGeoTransform ())
        
    #Need some vals for creating output extent etc
    #The GeoTransform object is:
    
    #GeoTransform[0] /* top left x */
    #GeoTransform[1] /* w-e pixel resolution */
    #GeoTransform[2] /* 0 */
    #GeoTransform[3] /* top left y */
    #GeoTransform[4] /* 0 */
    #GeoTransform[5] /* n-s pixel resolution (negative value) */
    
    #zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))
    #y_max is the SOUTHERN value! For us, that's MIN! Oh well...
    
    netCDFData = netCDF4.Dataset(theFile, 'r')
    #print(netCDFData)
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    #These need to be adjusted so that we get pixel edges shifted, leaving
    #our SILO coords mid-pixel
    pixWidth = 0.05
    
    #Source Gdal loading looks at this pixHeight value to work out the lower left coordinate
    #Either way, Source will interpret inGeoTransform[3] as being our North coordinate,
    #calculating the yllCorner to be a MORE NEGATIVE value.
    
    #xllCorner = inGeoTransform[0];
    #yllCorner = (inGeoTransform[5] > 0)?inGeoTransform[3] - ( inGeoTransform[5] * inDataset.RasterYSize ):
    #                                   inGeoTransform[3] + ( inGeoTransform[5] * inDataset.RasterYSize );
    
    pixHeight = pixWidth * -1
    #pixHeight = pixWidth
    
    #x_min = lons[:1] - (0.5 * pixWidth)
    x_min = min(lons) - (0.5 * pixWidth)
    
    #This gives first value, which in SILO is the smallest or 'most negative' or 'most southerly'
    #y_min = lats[:1] - (0.5 * pixHeight)
    
    #This gives last value, which in SILO is the largest or 'most positive' or 'most northerly'
    #y_min = lats[len(lats)-1:] - (0.5 * pixHeight)
    
    #-10
    y_min = max(lats) - (0.5 * pixHeight)
    
    #-44
    #yes, use lower left if supplying positive pixel height
    #y_min = min(lats) - (0.5 * pixHeight)
    
    
    netCDFData.close()
    
    dStart = datetime.date(int(year), 1, 1)
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)#WGS84

    theDataSet.SetProjection(srs.ExportToWkt())
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    for band in range( theDataSet.RasterCount ):
           
        dateAdd = band
        dateNow = dStart + datetime.timedelta(days=dateAdd)
        
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        
        band += 1
        #print("[ GETTING BAND ]: ", band)
        srcband = theDataSet.GetRasterBand(band)
        
        if srcband is None:
            continue
        
        scaleFactor = 1.0
        
        sf = srcband.GetMetadataItem(dataVarName + '_scale_factor')
        nd = srcband.GetMetadataItem(dataVarName + '__FillValue')
        #print(sf)
        
        scaleFactor = float(sf)
        rawNoData = float(nd)
        
        rasterOutputFile = thisYearDir + '/' + dateForFileName + fileNameEndBit
        outRaster = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, theDataSet.RasterXSize, theDataSet.RasterYSize, 1, gdal.GDT_Float32)
        outRaster.SetGeoTransform((x_min, pixWidth, 0, y_min, 0, pixHeight))
        outBand = outRaster.GetRasterBand(1)
        bandArray = srcband.ReadAsArray()
        
        #Why does gdal_grid turn image upside-down?
        #revert the array lines
        #A few things to note:
        #
        #The point used in the geotransform should be upper-left
        #The resolution in y-direction should be negative
        #In the projection (at least the ones I use) positive y-direction is up
        #In numpy arrays positive y-direction is down
        #When using gdal's WriteArray it uses the upper left corner
        #
        #Hope this helps other people's confusion.
        
        bandArray = bandArray[::-1,:]
        outBand.WriteArray(bandArray * scaleFactor)
        outBand.SetNoDataValue(rawNoData * scaleFactor)
        outRaster.SetProjection(srs.ExportToWkt())
        outBand.FlushCache()
    
    

print("All Done")


Current year: 1980
Processing: P:/projects/SILO/Wes/rain/1980.daily_rain.nc
Current year: 1981
Processing: P:/projects/SILO/Wes/rain/1981.daily_rain.nc
Current year: 1982
Processing: P:/projects/SILO/Wes/rain/1982.daily_rain.nc
Current year: 1983
Processing: P:/projects/SILO/Wes/rain/1983.daily_rain.nc
Current year: 1984
Processing: P:/projects/SILO/Wes/rain/1984.daily_rain.nc
Current year: 1985
Processing: P:/projects/SILO/Wes/rain/1985.daily_rain.nc
Current year: 1986
Processing: P:/projects/SILO/Wes/rain/1986.daily_rain.nc
Current year: 1987
Processing: P:/projects/SILO/Wes/rain/1987.daily_rain.nc
Current year: 1988
Processing: P:/projects/SILO/Wes/rain/1988.daily_rain.nc
Current year: 1989
Processing: P:/projects/SILO/Wes/rain/1989.daily_rain.nc
Current year: 1990
Processing: P:/projects/SILO/Wes/rain/1990.daily_rain.nc
Current year: 1991
Processing: P:/projects/SILO/Wes/rain/1991.daily_rain.nc
Current year: 1992
Processing: P:/projects/SILO/Wes/rain/1992.daily_rain.nc
Current year